In [50]:
import pandas as pd
import numpy as np

In [51]:
data = pd.read_csv("data.csv", parse_dates=['Data'])

In [52]:
data

,Data,Casa,Fora,Gols Casa,Gols Fora,Edição,Fase
0,2001-07-11,Equador,Chile,1.0,4.0,2001,Grupo A
1,2001-07-11,Colombia,Venezuela,2.0,0.0,2001,Grupo A
2,2001-07-14,Chile,Venezuela,1.0,0.0,2001,Grupo A
3,2001-07-14,Colombia,Equador,1.0,0.0,2001,Grupo A
4,2001-07-17,Equador,Venezuela,4.0,0.0,2001,Grupo A
...,...,...,...,...,...,...,...
207,2021-06-27,Venezuela,Peru,0.0,1.0,2021,Grupo B
208,2021-07-02,Peru,Paraguai,NaN,NaN,2021,Quartas
209,2021-07-02,Brasil,Chile,NaN,NaN,2021,Quartas
210,2021-07-03,Uruguai,Colombia,NaN,NaN,2021,Quartas


In [53]:
resultados = list()
for gc, gf in data[['Gols Casa', 'Gols Fora']].values:
    if gc > gf:
        resultados.append(1)
    elif gf > gc:
        resultados.append(-1)
    else:
        resultados.append(0)
data['resultados'] = resultados

In [54]:
ix = ["Edição","time"]
grupos = data[data['Fase'].str.contains("Grupo")]
newnames_casa = {"Casa": "time", -1: "derrota", 0:"empate", 1:"vitoria"}
newnames_fora = {"Fora": "time", -1: "vitoria", 0:"empate", 1:"derrota"}
grupos_casa = grupos.groupby(["Edição", "Casa", 'resultados']).size().unstack().fillna(0).reset_index().rename(columns=newnames_casa).set_index(ix)
grupos_fora = grupos.groupby(["Edição", "Fora", 'resultados']).size().unstack().fillna(0).reset_index().rename(columns=newnames_fora).set_index(ix)
partidas = grupos_casa.add(grupos_fora, fill_value = 0).reset_index()

In [55]:
ix = ["Edição","time"]
newnames_gcasa = {"Casa": "time", 'Gols Casa': "marcados", 'Gols Fora': "sofridos"}
gols_casa = grupos.groupby(["Edição", "Casa"])[['Gols Casa', 'Gols Fora']].sum().reset_index().rename(columns=newnames_gcasa).set_index(ix)
newnames_gfora = {"Fora": "time", 'Gols Casa': "sofridos", 'Gols Fora': "marcados"}
gols_fora = grupos.groupby(["Edição", "Fora"])[['Gols Casa', 'Gols Fora']].sum().reset_index().rename(columns=newnames_gfora).set_index(ix)
gols = gols_casa.add(gols_fora, fill_value=0).reset_index()

In [56]:
non_grupos = grupos = data[~data['Fase'].str.contains("Grupo")][['Edição','Casa', 'Fora', 'resultados']]

In [62]:
df = pd.merge(non_grupos, partidas, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df = pd.merge(df, partidas, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])
df = pd.merge(df, gols, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df = pd.merge(df, gols, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])

In [63]:
df.head()

,Edição,Casa,Fora,resultados,time_casa,derrota_casa,empate_casa,vitoria_casa,time_fora,derrota_fora,empate_fora,vitoria_fora,time_casa,marcados_casa,sofridos_casa,time_fora,marcados_fora,sofridos_fora
0,2001,Chile,Mexico,-1,Chile,1.0,0.0,2.0,Mexico,1.0,1.0,1.0,Chile,5.0,3.0,Mexico,1.0,1.0
1,2001,Costa Rica,Uruguai,-1,Costa Rica,0.0,1.0,2.0,Uruguai,1.0,1.0,1.0,Costa Rica,6.0,1.0,Uruguai,2.0,2.0
2,2001,Colombia,Peru,1,Colombia,0.0,0.0,3.0,Peru,1.0,1.0,1.0,Colombia,5.0,0.0,Peru,4.0,5.0
3,2001,Honduras,Brasil,1,Honduras,1.0,0.0,2.0,Brasil,1.0,0.0,2.0,Honduras,3.0,1.0,Brasil,5.0,2.0
4,2001,Mexico,Uruguai,1,Mexico,1.0,1.0,1.0,Uruguai,1.0,1.0,1.0,Mexico,1.0,1.0,Uruguai,2.0,2.0


In [64]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, classification_report, f1_score

In [70]:
noties = {6: -1,
 13: 1,
 15: -1,
 20:-1,
 25: -1,
 26: -1,
 29: 1,
 34: 1,
 35: -1,
 39: 1,
 41: -1,
 47: -1,
 48: 1,
 50: -1,
 51: -1
 
}
df.loc[df['resultados'] == 0, 'resultados'] = list(noties.values()) + [0]*4

In [71]:
df.loc[df['resultados'] == 0, 'resultados'].shape

(4,)

In [72]:
len(list(noties.values()))

15

In [73]:
df.loc[noties.keys()]

,Edição,Casa,Fora,resultados,time_casa,derrota_casa,empate_casa,vitoria_casa,time_fora,derrota_fora,empate_fora,vitoria_fora,time_casa,marcados_casa,sofridos_casa,time_fora,marcados_fora,sofridos_fora
6,2001,Uruguai,Honduras,-1,Uruguai,1.0,1.0,1.0,Honduras,1.0,0.0,2.0,Uruguai,2.0,2.0,Honduras,3.0,1.0
13,2004,Brasil,Uruguai,1,Brasil,1.0,0.0,2.0,Uruguai,1.0,1.0,1.0,Brasil,6.0,3.0,Uruguai,6.0,7.0
15,2004,Argentina,Brasil,-1,Argentina,1.0,0.0,2.0,Brasil,1.0,0.0,2.0,Argentina,10.0,4.0,Brasil,6.0,3.0
20,2007,Uruguai,Brasil,-1,Uruguai,1.0,1.0,1.0,Brasil,1.0,0.0,2.0,Uruguai,1.0,3.0,Brasil,4.0,2.0
25,2011,Argentina,Uruguai,-1,Argentina,0.0,2.0,1.0,Uruguai,0.0,2.0,1.0,Argentina,4.0,1.0,Uruguai,3.0,2.0
26,2011,Brasil,Paraguai,-1,Brasil,0.0,2.0,1.0,Paraguai,0.0,3.0,0.0,Brasil,6.0,4.0,Paraguai,5.0,5.0
29,2011,Paraguai,Venezuela,1,Paraguai,0.0,3.0,0.0,Venezuela,0.0,2.0,1.0,Paraguai,5.0,5.0,Venezuela,4.0,3.0
34,2015,Argentina,Colombia,1,Argentina,0.0,1.0,2.0,Colombia,1.0,1.0,1.0,Argentina,4.0,2.0,Colombia,1.0,1.0
35,2015,Brasil,Paraguai,-1,Brasil,1.0,0.0,2.0,Paraguai,0.0,2.0,1.0,Brasil,4.0,3.0,Paraguai,4.0,3.0
39,2015,Chile,Argentina,1,Chile,0.0,1.0,2.0,Argentina,0.0,1.0,2.0,Chile,10.0,3.0,Argentina,4.0,2.0


In [77]:
yval

8    -1
9     1
10   -1
11   -1
12    1
13    1
14   -1
15   -1
Name: resultados, dtype: int64

In [87]:
auc = list()
for e in [2004, 2007, 2011, 2015, 2016, 2019]:
    dftr = df[df['Edição'] < e]
    dfval = df[df['Edição'] == e]
    f = ['derrota_casa','empate_casa', 'vitoria_casa', 'derrota_fora',
       'empate_fora', 'vitoria_fora', 'marcados_casa',
       'sofridos_casa', 'marcados_fora', 'sofridos_fora']
    
    Xtr = dftr[f]
    ytr = dftr['resultados']
    Xval = dfval[f]
    yval = dfval['resultados']
    
    #print(dftr.shape)
    #print(dfval.shape)
    mdl = LogisticRegression(C=1, class_weight='balanced')
    #mdl = RandomForestClassifier(n_jobs=6, min_samples_leaf=1, n_estimators=100, random_state=0)
    mdl.fit(Xtr, ytr)
    
    
    p = mdl.predict_proba(Xval)[:,1]
    p_ = mdl.predict(Xval)
    auc_ = roc_auc_score(yval,p,multi_class='ovr')
    #f1 = 
    print("Ano: {} | LL: {} | AUC: {}".format(e, log_loss(yval, p), auc_))
    auc.append(auc_)
    #print(classification_report(yval, p_))
    print()
print(np.mean(auc))
    

Ano: 2004 | LL: 1.226978074392431 | AUC: 0.39999999999999997

Ano: 2007 | LL: 0.5536356669896119 | AUC: 0.8666666666666667

Ano: 2011 | LL: 1.5185050293366809 | AUC: 0.2666666666666666

Ano: 2015 | LL: 0.7077194776373048 | AUC: 0.5

Ano: 2016 | LL: 0.7579356078854942 | AUC: 0.9166666666666667

Ano: 2019 | LL: 0.6777662982798773 | AUC: 0.75

0.6166666666666667


In [88]:
p_

array([ 1, -1,  1,  1,  1,  1, -1,  1])

In [90]:
dfval['p'] = p_

<ipython-input-90-ada5b62984d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfval['p'] = p_


In [93]:
dfval[['Edição', 'Casa', 'Fora', 'resultados', 'p']]

,Edição,Casa,Fora,resultados,p
48,2019,Brasil,Paraguai,1,1
49,2019,Venezuela,Argentina,-1,-1
50,2019,Colombia,Chile,-1,1
51,2019,Uruguai,Peru,-1,1
52,2019,Brasil,Argentina,1,1
53,2019,Chile,Peru,-1,1
54,2019,Argentina,Chile,1,-1
55,2019,Brasil,Peru,1,1


# 2021

In [96]:
e = 2021
dftr = df[df['Edição'] < e]
dfval = df[df['Edição'] == e]
f = ['derrota_casa','empate_casa', 'vitoria_casa', 'derrota_fora',
   'empate_fora', 'vitoria_fora', 'marcados_casa',
   'sofridos_casa', 'marcados_fora', 'sofridos_fora']

Xtr = dftr[f]
ytr = dftr['resultados']
Xval = dfval[f]
yval = dfval['resultados']

#print(dftr.shape)
#print(dfval.shape)
mdl = LogisticRegression(C=1, class_weight='balanced')
#mdl = RandomForestClassifier(n_jobs=6, min_samples_leaf=1, n_estimators=100, random_state=0)
mdl.fit(Xtr, ytr)


p = mdl.predict_proba(Xval)[:,1]
p_ = mdl.predict(Xval)

    

In [97]:
p

array([0.07847179, 0.80576171, 0.47510932, 0.76395321])

In [100]:
dfval['p'] = p_

<ipython-input-100-ada5b62984d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfval['p'] = p_


In [101]:
dfval[['Edição', 'Casa', 'Fora', 'resultados', 'p']]

,Edição,Casa,Fora,resultados,p
56,2021,Peru,Paraguai,0,-1
57,2021,Brasil,Chile,0,1
58,2021,Uruguai,Colombia,0,-1
59,2021,Argentina,Equador,0,1


# Sim

In [106]:
semi = pd.DataFrame([{"Edição": 2021, "Casa": "Brasil", "Fora": "Paraguai", "resultados": 0},
 {"Edição": 2021, "Casa": "Argentina", "Fora": "Colombia", "resultados": 0}])
semi

,Edição,Casa,Fora,resultados
0,2021,Brasil,Paraguai,0
1,2021,Argentina,Colombia,0


In [107]:
df2 = pd.merge(semi, partidas, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df2 = pd.merge(df2, partidas, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])
df2 = pd.merge(df2, gols, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df2 = pd.merge(df2, gols, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])

In [110]:
df2['p'] = mdl.predict(df2[f])
df2[['Edição', 'Casa', 'Fora', 'resultados', 'p']]

,Edição,Casa,Fora,resultados,p
0,2021,Brasil,Paraguai,0,1
1,2021,Argentina,Colombia,0,1


In [112]:
final = pd.DataFrame([{"Edição": 2021, "Casa": "Argentina", "Fora": "Brasil", "resultados": 0}])
final

,Edição,Casa,Fora,resultados
0,2021,Argentina,Brasil,0


In [113]:
df3 = pd.merge(final, partidas, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df3 = pd.merge(df3, partidas, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])
df3 = pd.merge(df3, gols, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df3 = pd.merge(df3, gols, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])

In [114]:
df3['p'] = mdl.predict(df3[f])
df3[['Edição', 'Casa', 'Fora', 'resultados', 'p']]

,Edição,Casa,Fora,resultados,p
0,2021,Argentina,Brasil,0,1


In [116]:
mdl.predict_proba(df3[f])

array([[0.49730729, 0.50269271]])

In [132]:
possible = dfval['Casa'].tolist() + dfval['Fora'].tolist()
from itertools import permutations
df4 = pd.DataFrame(list(permutations(possible, 2)), columns=['Casa','Fora'])
df4['Edição'] = 2021
df4['resultados'] = 2021
df4 = pd.merge(df4, partidas, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df4 = pd.merge(df4, partidas, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])
df4 = pd.merge(df4, gols, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df4 = pd.merge(df4, gols, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])

In [138]:
df4['p'] = mdl.predict(df4[f])
df4 = df4[['Edição', 'Casa', 'Fora', 'p']]

In [141]:
finalr = list()
for e,c,fr,pp in df4.values:
    if pp == 1:
        finalr.append(c)
    else:
        finalr.append(fr)
df4['Vencedor'] = finalr

<ipython-input-141-25b95d4a0081>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Vencedor'] = finalr


In [143]:
df4[["Casa", "Fora", "Vencedor"]]

,Casa,Fora,Vencedor
0,Peru,Brasil,Brasil
1,Peru,Uruguai,Uruguai
2,Peru,Argentina,Argentina
3,Peru,Paraguai,Paraguai
4,Peru,Chile,Chile
5,Peru,Colombia,Colombia
6,Peru,Equador,Equador
7,Brasil,Peru,Peru
8,Brasil,Uruguai,Brasil
9,Brasil,Argentina,Brasil
